In [21]:
import numpy as np
import random
import matplotlib.pyplot as plt
import xarray as xr

In [22]:
#Hourly 2D bottom currents
#jan_jun = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jan_jun.nc")
jun_aug = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_jun-aug.nc")
#sep_dec = xr.open_dataset("/cluster/projects/itk-SINMOD/coral-mapping/midnor/samp_2D_sep-dec.nc")

In [16]:
print(len(jan_jun.variables['time']))

4201


In [17]:
#Remove overlap
jan_jun = jan_jun.isel(time=slice(0, -360))  
jun_aug = jun_aug.isel(time=slice(0, -192))  

In [ ]:
#Merge (not possible with this memory size.. Can do this later if necessary)
current_2019 = xr.concat([current_jan_jun, current_jun_aug, current_sep_dec], dim="time")

In [23]:
#Importing coral data
import pandas as pd

coral_data = pd.read_parquet('/cluster/home/maikents/midnor_total_coral_data_processed_v2.parquet')


In [19]:
#Histogram for bottom current speed
hor_res = jun_aug.grid_mapping.attrs['horizontal_resolution']
t_start, t_stop = (0,1)#(0, len(jun_aug.variables['time']))
coral_values_jun_aug = []
valid_coords_jun_aug = []
xc_max = jun_aug.variables['xc'].shape[0]
yc_max = jun_aug.variables['yc'].shape[0]
for time in range(t_start, t_stop):
        for x, y in zip(coral_data['x']/hor_res, coral_data['y']/hor_res):
            x = np.clip(np.round(coral_data['x']/hor_res).astype(int), 0, xc_max-1)
            y = np.clip(np.round(coral_data['y']/hor_res).astype(int), 0, yc_max-1)
            value = np.sqrt(jun_aug.variables['u-bottom'][time, y, x]**2 + jun_aug.variables['v-bottom'][time, y, x]**2)
            if not np.ma.is_masked(value):
                coral_values_jun_aug.append(value)
    
                    


In [24]:
import time
import numpy as np

hor_res = jun_aug.grid_mapping.attrs['horizontal_resolution']
t_start, t_stop = (0, 1)  # (0, len(jun_aug.variables['time']))
coral_values_jun_aug = []
valid_coords_jun_aug = []
xc_max = jun_aug.variables['xc'].shape[0]
yc_max = jun_aug.variables['yc'].shape[0]

timing_results = []

start_total = time.perf_counter()  # Track total execution time

print(f"Processing from time index {t_start} to {t_stop-1}")
print(f"Grid size: xc_max={xc_max}, yc_max={yc_max}")

for time_idx in range(t_start, t_stop):
    start_time = time.perf_counter()
    print(f"\nProcessing timestep {time_idx}...")

    # Precompute x and y indices
    x_indices = np.clip(np.round(coral_data['x'] / hor_res).astype(int), 0, xc_max - 1)
    y_indices = np.clip(np.round(coral_data['y'] / hor_res).astype(int), 0, yc_max - 1)

    print(f"Total coral points: {len(x_indices)}")
    
    for i, (x, y) in enumerate(zip(x_indices, y_indices)):
        step_start = time.perf_counter()

        # Check if indices are within bounds
        if 0 <= x < xc_max and 0 <= y < yc_max:
            value = np.sqrt(jun_aug.variables['u-bottom'][time_idx, y, x]**2 +
                            jun_aug.variables['v-bottom'][time_idx, y, x]**2)
            
            print(f"Point {i}: (x={x}, y={y}) -> Current Speed: {value:.6f}")

            if not np.ma.is_masked(value):
                coral_values_jun_aug.append(value)
                print(f"Value at ({x}, {y}) added: {value:.6f}")
            else:
                print(f"Skipping masked value at ({x}, {y})")
        else:
            print(f"Skipping out-of-bounds point: x={x}, y={y}")

        step_end = time.perf_counter()
        timing_results.append(f"Time step {time_idx}, Point {i} (x={x}, y={y}) - {step_end - step_start:.6f} sec")

    end_time = time.perf_counter()
    print(f"Finished timestep {time_idx} in {end_time - start_time:.6f} seconds")

end_total = time.perf_counter()
print(f"\nTotal execution time: {end_total - start_total:.6f} seconds")

# Optional: Print or save timing results
for result in timing_results[:10]:  # Print only first 10 to avoid too much output
    print(result)

Processing from time index 0 to 0
Grid size: xc_max=950, yc_max=555

Processing timestep 0...
Total coral points: 941
Point 0: (x=705, y=179) -> Current Speed: 0.045082
Value at (705, 179) added: 0.045082
Point 1: (x=612, y=44) -> Current Speed: nan
Value at (612, 44) added: nan
Point 2: (x=528, y=176) -> Current Speed: 0.037903
Value at (528, 176) added: 0.037903
Point 3: (x=637, y=157) -> Current Speed: 0.086152
Value at (637, 157) added: 0.086152
Point 4: (x=586, y=182) -> Current Speed: 0.080927
Value at (586, 182) added: 0.080927
Point 5: (x=744, y=138) -> Current Speed: 0.051348
Value at (744, 138) added: 0.051348
Point 6: (x=582, y=113) -> Current Speed: 0.058280
Value at (582, 113) added: 0.058280
Point 7: (x=631, y=169) -> Current Speed: 0.049441
Value at (631, 169) added: 0.049441
Point 8: (x=584, y=71) -> Current Speed: nan
Value at (584, 71) added: nan
Point 9: (x=571, y=179) -> Current Speed: 0.028864
Value at (571, 179) added: 0.028864
Point 10: (x=537, y=183) -> Current 

In [20]:
#Convert list to numpy array for easier manipulation
coral_values_jun_aug = np.array(coral_values_jun_aug)

#Flatten the array to combine all time steps
coral_values_jun_aug_flat = coral_values_jun_aug.flatten()


In [ ]:
np.savetxt("coral_values_jun_aug.csv", coral_values_jun_aug, delimiter=",")
pd.savecsv()

In [ ]:
# Calculate statistics for coral values
stats_jun_aug = {
    'mean': np.mean(coral_values_jun_aug_flat),
    'max': np.max(coral_values_jun_aug_flat),
    'min': np.min(coral_values_jun_aug_flat,
    '90th_percentile': np.percentile(coral_values_jun_aug_flat, 90),
    '10th_percentile': np.percentile(coral_values_jun_aug_flat, 10)
}

print(f"Statistics for coral values: {stats_jun_aug}")


In [ ]:
#Determine the bin edges for the histograms
bin_edges = np.histogram_bin_edges(coral_values_jun_aug_flat, bins=20)

In [ ]:
np.savetxt("bin_edges.csv", bin_edges, delimiter=",")


In [ ]:
#Plot histogram of the combined variable values
plt.hist(coral_values_jun_aug_flat, bins=bin_edges, edgecolor='black')
plt.xlabel('Current speed [m/s]', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title(f'Histogram of current speeds at coral data points', fontsize=16)
plt.grid(True)
plt.savefig(f'/cluster/home/maikents/current_speed_histogram_coral_points_jun_aug.png')
    
plt.show()